In [1]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from apollo.settings import LONG_SIGNAL, NO_SIGNAL, SHORT_SIGNAL
from apollo.utils.common import to_default_date_string

In [2]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.average_true_range as atr
import apollo.calculations.chaikin_accumulation_distribution as cad
import apollo.calculations.hull_moving_average as hma
import apollo.calculations.keltner_channel as kc

importlib.reload(yac)
importlib.reload(hma)
importlib.reload(kc)
importlib.reload(atr)
importlib.reload(cad);

In [3]:
ticker = "SPY"
start_date = "2023-06-01"
end_date = "2024-01-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

[*********************100%%**********************]  1 of 1 completed


In [4]:
WINDOW_SIZE = 20

hma_calculator = hma.HullMovingAverageCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)
hma_calculator.calculate_hull_moving_average()

atr_calculator = atr.AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)
atr_calculator.calculate_average_true_range()

kc_calculator = kc.KeltnerChannelCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
    volatility_multiplier=1.1,
)
kc_calculator.calculate_keltner_channel()

cad_calculator = cad.ChaikinAccumulationDistributionCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)
cad_calculator.calculate_chaikin_accumulation_distribution_line()

dataframe.dropna(inplace=True)

dataframe["signal"] = NO_SIGNAL

long = (dataframe["adj close"] > dataframe["lkc_bound"]) & (
    dataframe["adl"] > dataframe["prev_adl"]
)
dataframe.loc[long, "signal"] = LONG_SIGNAL

short = (dataframe["adj close"] < dataframe["ukc_bound"]) & (
    dataframe["adl"] < dataframe["prev_adl"]
)
dataframe.loc[short, "signal"] = SHORT_SIGNAL

dataframe

,ticker,open,high,low,close,adj close,volume,hma,tr,atr,lkc_bound,ukc_bound,adl,prev_adl,signal
date,,,,,,,,,,,,,,,
2023-07-27,SPY,459.019989,459.440002,451.549988,452.489990,446.248138,92194400,450.633511,7.890015,3.387054,446.907752,454.359270,1.209467e+08,2.143889e+08,-1
2023-07-28,SPY,455.880005,457.779999,452.489990,456.920013,450.617035,80011800,450.789504,5.290009,3.482201,446.959082,454.619925,1.154855e+08,1.209467e+08,-1
2023-07-31,SPY,457.410004,458.160004,456.049988,457.790009,451.475037,62040400,451.063089,2.110016,3.413592,447.308137,454.818040,1.179430e+08,1.154855e+08,1
2023-08-01,SPY,456.269989,457.250000,455.489990,456.480011,450.183136,55291500,451.262665,2.300018,3.357913,447.568960,454.956369,1.051785e+08,1.179430e+08,-1
2023-08-02,SPY,453.250000,453.519989,449.350006,450.130005,443.920715,93933400,450.745788,7.130005,3.546518,446.844618,454.646957,3.258856e+07,1.051785e+08,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,SPY,473.859985,475.380005,471.700012,473.649994,470.664490,67126600,472.110392,3.679993,4.328476,467.349068,476.871716,4.332101e+08,4.434859e+08,-1
2023-12-26,SPY,474.070007,476.579987,473.989990,475.649994,472.651886,55387000,473.110091,2.929993,4.258552,468.425684,477.794499,4.706698e+08,4.332101e+08,1
2023-12-27,SPY,475.440002,476.660004,474.890015,476.510010,473.506470,68000300,474.065575,1.769989,4.134124,469.518038,478.613111,5.251270e+08,4.706698e+08,1


In [5]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["adj close"],
        line={"color": "blue", "width": 1},
        name="Adj close",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["lkc_bound"],
        line={"color": "red", "width": 1},
        name="Lower KC",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["ukc_bound"],
        line={"color": "red", "width": 1},
        name="Upper KC",
    ),
    row=1,
    col=1,
)

fig.update_layout(
    title={"text": f"{ticker}", "x": 0.5},
    height=650,
    plot_bgcolor="#EFF5F8",
)

fig.update_yaxes(visible=False, secondary_y=True)

In [6]:
prime_value = "close"
trace_value = "adl"

x = dataframe.index.to_numpy()

y1 = dataframe[prime_value].to_numpy()
y2 = dataframe[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{ticker}:"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)